In [1]:
!pip install datasets -q

In [48]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn import *

from datasets import load_dataset

In [2]:
dataset = load_dataset("BAAI/TACO")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for BAAI/TACO contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/BAAI/TACO
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/25443 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [34]:
sample = next(iter(dataset['train']))

In [35]:
print(f'Keys: {list(sample.keys())}')
print(f"Count of Keys: {len(sample.keys())}")

Keys: ['question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity']
Count of Keys: 17


In [36]:
print(sample['question'])

This is an interactive problem.

In good old times dwarves tried to develop extrasensory abilities:

  * Exactly n dwarves entered completely dark cave. 
  * Each dwarf received a hat — white or black. While in cave, none of the dwarves was able to see either his own hat or hats of other Dwarves. 
  * Dwarves went out of the cave to the meadow and sat at an arbitrary place one after the other. When a dwarf leaves the cave, he sees the colors of all hats of all dwarves that are seating on the meadow (i.e. left the cave before him). However, he is not able to see the color of his own hat and none of the dwarves can give him this information. 
  * The task for dwarves was to got diverged into two parts — one with dwarves with white hats and one with black hats. 



After many centuries, dwarves finally managed to select the right place on the meadow without error. Will you be able to repeat their success?

You are asked to successively name n different integer points on the plane. After n

In [38]:
sample['solutions']

'[]'

In [47]:
trainData = dataset['train'].to_pandas()
testData = dataset['test'].to_pandas()

In [44]:
trainData

,question,solutions,starter_code,input_output,difficulty,raw_tags,name,source,tags,skill_types,url,Expected Auxiliary Space,time_limit,date,picture_num,memory_limit,Expected Time Complexity
0,This is an interactive problem.\n\nIn good old...,[],,"{""inputs"": [""hack\n30\n1 0 1 1 1 1 1 1 1 1 1 1...",HARD,"['interactive', 'binary search', 'geometry', '...",None,codeforces,"['Geometry', 'Sorting', 'Constructive algorith...",['Sorting'],https://codeforces.com/problemset/problem/1063/C,None,2.0 seconds,None,None,256.0 megabytes,None
1,There are $n$ candy boxes in front of Tania. T...,"[""INF = 10000000000.0\nmax_n = 50\nmax_k = 200...",,"{""inputs"": [""5 3 10\n1 2 3 4 5\nRGBRR\n"", ""2 1...",HARD,['dp'],None,codeforces,['Dynamic programming'],['Dynamic programming'],https://codeforces.com/problemset/problem/1057/C,None,None,2019-12-31,None,None,None
2,Little Petya likes to play a lot. Most of all ...,[],,"{""inputs"": [""10 10\n5 1 2 4 1 7 3 8 10 8\n0 5 ...",VERY_HARD,"['data structures', 'dsu']",None,codeforces,"['Spanning trees', 'Data structures']",['Data structures'],https://codeforces.com/problemset/problem/13/E,None,1.0 seconds,None,None,64.0 megabytes,None
3,"If you visit Aizu Akabeko shrine, you will fin...","[""def sub(maxs, mins):\n\tfor i in range(len(m...",,"{""inputs"": [""9714431"", ""16612328"", ""23422731"",...",UNKNOWN_DIFFICULTY,[],None,aizu,[],[],None,None,1.0 seconds,None,None,268.435456 megabytes,None
4,"You have a deck of $n$ cards, and you'd like t...","[""import heapq\nfrom math import sqrt\nimport ...",,"{""inputs"": [""4\n4\n1 2 3 4\n5\n1 5 2 4 3\n6\n4...",EASY,"['data structures', 'greedy', 'math']",None,codeforces,"['Data structures', 'Mathematics', 'Greedy alg...","['Data structures', 'Greedy algorithms']",https://codeforces.com/problemset/problem/1492/B,None,1 second,2021-02-23,0,512 megabytes,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25438,The manager of the Japanese sweets shop Tomogu...,[],,"{""inputs"": [""5\n4 9 1 3 8\n10\n8 5 3 6 2 1 4 5...",UNKNOWN_DIFFICULTY,[],None,aizu,[],[],None,None,8.0 seconds,None,None,134.217728 megabytes,None
25439,"Your game development studio, Ad Hoc Entertain...",[],,"{""inputs"": [""20 30\n2 10 20 2 24 18 28 10\n"", ...",HARD,[],None,kattis,[],[],https://open.kattis.com/problems/trickshot,None,None,None,None,None,None
25440,#Sorting on planet Twisted-3-7\n\nThere is a p...,"[""def sort_twisted37(arr):\n\n\tdef key(x):\n\...",def sort_twisted37(arr):\n\t,"{""fn_name"": ""sort_twisted37"", ""inputs"": [[[1, ...",EASY,"['Mathematics', 'Arrays', 'Algorithms', 'Sorti...",None,codewars,"['Sorting', 'Data structures', 'Mathematics']","['Sorting', 'Data structures']",https://www.codewars.com/kata/58068479c27998b1...,None,None,None,None,None,None
25441,Optimistan is a strange country. It is situate...,[],,"{""inputs"": [""3\n4 4\n2\n"", ""4\n2 2 2\n2 2\n2\n...",HARD,[],None,kattis,[],[],https://open.kattis.com/problems/optimistan,None,None,None,None,None,None
